# LEPL1202 - APP1 - Partie B

Bienvenue sur le Notebook interactif de l'APP1 - Partie B.

## Qu'est qu'un Notebook ?

Les Notebooks (aussi appelés Jupyter Notebooks) sont des documents permettant combiner code et du texte (Markdown), le tout dans des cellules individuelles. Par example, cette cellule est entièrement constituée de texte et la prochaine est une cellule de code (Python). Pour les plus curieux, nous vous invitons à aller sur [leur site](https://jupyter.org/).

Pour modifier une cellule, il suffit de double-cliquer dessus. Pour exécuter une cellule, il existe plusieurs façons de faire:

1. cliquer sur <span class="kb"><kbd>Shift-Enter</kbd></span>;
2. cliquer sur l'icône <i class="fa-step-forward fa"></i> (*"run this cell"*), à gauche d'une cellule;
3. cliquer sur l'icône <i class="fa-play fa"></i><span class="toolbar-btn-label">Run</span>;
4. ou choisir l'une des options du menu `Cell` (tout en haut).

<b>Attention :</b> même si les cellules de code s'exécutent individuellement, vous pouvez réutiliser des fonctions ou variables définies dans une cellule partout après. Notez que l'ordre d'exécution cellules est pris en compte. Ceci est indiqué par le numéro entre crochet (exemple : <bdi>In</bdi>&nbsp;[1]). Si vous avez un doute sur la bonne exécution de vos cellules, et que quelque chose ne fonctionne pas comme attendu, cliquez sur le bouton <i class="fa-forward fa"></i> et *"Restart and Run All Cells"*.

## Que dois-je faire ?

Ce notebook doit être lu et complété en parallèle de l'énoncé PDF.

La plupart de temps, il vous faudra juste **lire** et **intéragir** avec les quelques boutons / glissières qui rendent les graphiques interactifs.

Quand du code de votre part est attendu, il sera indiqué avec des commentaires.

<b>Note :</b> certaines cellules de code ont été volontairement cachées, pour gagner en visibilité. Il faut cependant bien les exécuter (<i class="fa-step-forward fa"></i>). Pour afficher leur contenu, dans le menu, `View -> Cell Toolbar -> Hide code`.

## Objectif

L'objectif de ce Notebook est de vous faire valider de manière numérique les concepts vu lors de l'APP1 - Partie B. Le but n'est en aucun cas de vous apprendre à faire des méthodes numériques (ceci sera le sujet du cours LEPL1104), et c'est pourquoi il y a très peu de code à fournir de votre part.

<hr>

# 1. Import des modules nécessaires

Comme dans beaucoup de projets Python, nous allons utiliser des modules externes.

Parmi ceux-ci, celui avec lequel vous allez intéragir se nomme [**NumPy**](https://numpy.org/). Par convention, il est importé sous l'alias `np`:

```python
import numpy as np
```

**NumPy** est un module permettant d'effectuer des opérations mathématiques classiques sur des vecteurs / matrices multi-dimensionnelles, de manière très performante.

Dès lors, la plupart des fonctions usuelles se retrouvent dans **NumPy**:

* `math.cos(x)` devient `np.cos(x)`;
* la somme des valeurs d'un vecteur `x` s'obtient avec `np.sum(x)`;
* le produit classique (1 à 1) de `x` avec `y` se fait via `x * y`;
* le produit matriciel de `x` avec `y` se fait via `x @ y`;
* les dimensions d'une matrice `x` sont données par l'attribut `x.shape`;
* le produit scalaire (*"dot-product"*) de `x` et `y` se fait via `np.dot(x, y)`;
* et bien plus, voir [leur guide](https://numpy.org/devdocs/user/quickstart.html).

Cliquez sur <i class="fa-step-forward fa"></i> pour importer tous les modules dont nous allons avoir besoin.

In [ ]:
import uuid

import numpy as np
import plotly.graph_objects as go

from ipywidgets import interact
from typing import Tuple, Union

<hr>

# 2. Définition des courbes Rouge et Bleue

Dans la cellule suivante, nous avons définis les courbes Rouge et Bleue, faisant références aux courbes homonymes de l'énoncé. Ces deux fonctions prennent un vecteur **NumPy** (`np.ndarray`) comme paramètre pour les abscisses, et des nombre à virgules (`float`) pour les autres.

Cliquez sur <i class="fa-step-forward fa"></i> pour définir ces deux fonctions.

In [ ]:
def R(
    x: np.ndarray,
    z_a: float = 1,
    z_b: float = 0,
    L: float = 1,
    return_dydx: bool = False,
) -> Union[Tuple[np.ndarray, np.ndarray]]:
    """
    Courbe R(ouge), évaluée aux points x.

    x: vecteur des abscisses, de 0 à L, de longueur n
    z_a, z_b, L: paramètres de la courbe
    return_dydx: si True, retourne aussi la dérivée première de la courbe
    """
    y = z_b + (z_a - z_b) * ((x - L) / L) ** 2

    if return_dydx:
        dydx = (2 / L) * (z_a - z_b) * ((x - L) / L)
        return y, dydx
    else:
        return y


def B(
    x: np.ndarray,
    z_a: float = 1,
    z_b: float = 0,
    L: float = 1,
    return_dydx: bool = False,
) -> Union[Tuple[np.ndarray, np.ndarray]]:
    """
    Courbe B(leue), évaluée aux points x.

    x: vecteur des abscisses, de 0 à L, de longueur n
    z_a, z_b, L: paramètres de la courbe
    return_dydx: si True, retourne aussi la dérivée première de la courbe
    """
    y = z_a - (z_a - z_b) * (x / L)

    if return_dydx:
        dydx = (z_b - z_a) * np.ones_like(x) / L
        return y, dydx
    else:
        return y

Pour visualiser les courbes, et l'impact de paramètres optionnels, cliquez sur <i class="fa-step-forward fa"></i> pour afficher un graphique interactif.

Vous pouvez faire varier la valeur de chacun des paramètres en utilisant les glissières. Le graphique se mettra à jour automatiquement.

In [ ]:
trace_R = go.Scatter(
    x=[],
    y=[],
    fill="tozeroy",
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=[],
    y=[],
    fill="tonexty",
    name="B",
    mode="lines",
    line_color="blue",
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_1 = go.FigureWidget(data=[trace_R, trace_B, trace_Z])


@interact(L=(1, 10, 1), z_a=(0, 100, 1), z_b=(0, 100, 1))
def update(L=1, z_a=1, z_b=0):
    with fig_1.batch_update():
        x = np.linspace(0, L)
        fig_1.data[0].x = x
        fig_1.data[0].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_1.data[1].x = x
        fig_1.data[1].y = B(x, L=L, z_a=z_a, z_b=z_b)
        fig_1.data[2].x = [0, L]
        fig_1.data[2].y = [z_a, z_b]
        fig_1.data[2].uid = str(uuid.uuid4())  # This forces the text to be updated


fig_1

<hr>

# 3. Aire sous la courbe

De manière assez évidente, l'aire sous la courbe n'est pas la même pour la courbe rouge que pour la courbe bleue.

Nous vous demandons d'estimer l'aire sous ces courbes via une somme de Riemann.

Pour rappel, une estimation simple de l'aire sous une courbe peut se faire en sommant l'aire de plusieurs rectangles :

* si $x = [0, \Delta x, 2 \Delta x, ..., L]$, le vecteur abscisses avec $n$ points;
* et $y = f(x)$, avec $f=R$ ou $f=B$, en fonction la courbe;
* alors, l'aire sous la courbe peut être estimée comme valant

$$ \text{A} \approx \sum\limits_{i=0}^{i \lt n - 1} y_i \cdot \Delta x = \Delta x \sum\limits_{i=0}^{i \lt n - 1} y_i.$$

Question bonus : pourquoi $n-1$ et pas $n$ dans la somme ?

Notez qu'au lieu d'utiliser des rectangles, vous pouvez utiliser des trapèzes pour avoir une estimation plus précise. Pour ce faire, nous vous conseillons d'utiliser la fonction [`np.trapz`](https://numpy.org/doc/stable/reference/generated/numpy.trapz.html).

In [ ]:
def aire(x: np.ndarray, y: np.ndarray) -> float:
    """
    Calcule l'aire sous la courbe, en utilisant l'intégrale
    par somme des rectangles (ou [BONUS] trapèzes).

    x: vecteur abscisses, de 0 à L, de longueur n
    y: vecteur ordonnées, de longueur n
    """
    # TODO : modifier le code pour retourner une estimation correcte de l'aire
    dx = x[1] - x[0]  # Base d'un rectangle

    return 0  # À changer !

Une fois la fonction `aire` complétée, cliquez sur <i class="fa-step-forward fa"></i> pour la cellule ci-dessus, et celle ci-dessous. Si votre code est correct, la valeur d'aire affichée devrait s'approcher de la vraie valeur au plus `n` est grand.

Si vous avez utlisé la méthode des trapèzes, vous devriez avoir l'aire exacte de la courbe bleue, quelque soit `n`.

Question bonus : pourquoi ?

In [ ]:
trace_R = go.Scatter(
    x=[],
    y=[],
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=[],
    y=[],
    name="B",
    mode="lines",
    line_color="blue",
)

trace_R_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="red",
    name="Aire approx. de R",
    opacity=0.5,
)

trace_B_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="blue",
    name="Aire approx. de B",
    opacity=0.5,
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_2 = go.FigureWidget(
    data=[trace_B_approx, trace_R_approx, trace_R, trace_B, trace_Z]
)


@interact(L=(1, 10, 1), z_a=(0, 100, 1), z_b=(0, 100, 1), n=(4, 100, 4))
def update(L=1, z_a=1, z_b=0, n=4):
    with fig_2.batch_update():
        x = np.linspace(0, L)
        fig_2.data[2].x = x
        fig_2.data[2].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_2.data[3].x = x
        fig_2.data[3].y = B(x, L=L, z_a=z_a, z_b=z_b)
        x, dx = np.linspace(0, L, n, retstep=True)
        y_R, dydx_R = R(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        y_B, dydx_B = B(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        fig_2.data[1].x = x[:-1]
        fig_2.data[1].y = y_R[:-1]
        fig_2.data[1].width = dx
        fig_2.data[0].x = x[:-1]
        fig_2.data[0].y = y_B[:-1]
        fig_2.data[0].width = dx
        fig_2.data[-1].x = [0, L]
        fig_2.data[-1].y = [z_a, z_b]
        fig_2.data[-1].uid = str(uuid.uuid4())  # This forces the text to be updated
        aire_R = aire(x, y_R)
        aire_B = aire(x, y_B)
        fig_2.layout.title.text = f"Aire R : {aire_R:.4} / Aire B : {aire_B:.4}"


fig_2

<hr>

# 4. Travail d'une force

L'objectif de cette partie est de calculer le travail effectué par la force $\vec{F}$ le long du chemin.

Dans le cas de la force de gravité, nous nous attendrions à avoir le même résultat quelque soit la courbe, conséquence du champ conservatif. Cependant, nous voyons bien que l'aire sous la courbe est différente : le travail n'est donc pas directement proportionnel à l'aire !


Pour calculer le travail, il faut donc effectuer l'intégrale :

$$ W = \int\limits_{0}^{L} \vec{F}\cdot d\vec{\ell},$$

avec $\vec{\ell}=\begin{pmatrix}1 & dy/dx\end{pmatrix}^\intercal$ et $d\vec{\ell}=\vec{\ell} \cdot dx$.

Pour ce faire, vous devez compléter la fonction `travail`, où la matrice de vecteurs de taille $n \times 2$, $\vec{\ell}$, a déjà été définie. Cette dernière contient, pour chaque abscisse $x_i$, le vecteur tangent $\vec{\ell}_i$.

Afin de compléter ce code, il peut vous être utile d'utiliser la fonction `dot2` (voir ci-dessous) et votre précédente fonction `aire`.

In [ ]:
def dot2(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Calule le produit scalaire entre deux "listes" de n vecteurs
    de longueur m (= 2 ou 3 en pratique).

    Ceci est équivalent mais plus rapide que de faire une boucle `for`
    sur chaque ligne i et de calculer le produit scalaire entre deux
    vecteurs a[i, :] et b[i, :].

    a: une matrice de vecteurs, de dimension n x m
    b: une matrice de vecteurs, de dimension n x m
    """
    return np.sum(a * b, axis=1)


def force(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    """
    Evalue le vecteur force appliqué à un ou plusieurs points (x, y).

    x: vecteur abscisses, de 0 à L, de longueur n
    y: vecteur ordonnées, de longueur n
    """
    g = 9.81  # m / s^2
    m = 1  # kg
    n = len(x)

    F_x = np.zeros(n)  # pas de force selon x
    F_y = -m * np.ones(n) * g
    return np.column_stack([F_x, F_y])


def travail(x: np.ndarray, y: np.ndarray, dydx: np.ndarray, F: np.ndarray) -> float:
    """
    Calcule le travail effectué par F le long d'une courbe.

    x: vecteur abscisses, de 0 à L, de longueur n
    y: vecteur ordonnées, de longueur n
    dydx: vecteur des dérivées % à x, de longueur n
    F: matrice des forces, de dimension n x 2, i.e., F[i, :] = [F_x_i, F_y_i]
    """
    n = len(x)
    l = np.zeros((n, 2))  # vecteur tangent l
    l[:, 0] = np.ones(n)
    l[:, 1] = dydx
    # Ne pas modifier ce qui est au dessus
    
    W = ...

    return 0  # À changer !

Une fois la fonction `travail` complétée, cliquez sur <i class="fa-step-forward fa"></i> pour la cellule ci-dessus, et celle ci-dessous. Si votre code est correct, les deux valeurs de travail devraient se rapprocher de $m\cdot g \cdot (z_a - z_b)$ au plus `n` est grand.

In [ ]:
trace_R = go.Scatter(
    x=[],
    y=[],
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=[],
    y=[],
    name="B",
    mode="lines",
    line_color="blue",
)

trace_R_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="red",
    name="Aire approx. de R",
    opacity=0.5,
)

trace_B_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="blue",
    name="Aire approx. de B",
    opacity=0.5,
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_3 = go.FigureWidget(
    data=[trace_B_approx, trace_R_approx, trace_R, trace_B, trace_Z]
)


@interact(L=(1, 10, 1), z_a=(0, 100, 1), z_b=(0, 100, 1), n=(4, 100, 4))
def update(L=1, z_a=1, z_b=0, n=4):
    with fig_3.batch_update():
        x = np.linspace(0, L)
        fig_3.data[2].x = x
        fig_3.data[2].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_3.data[3].x = x
        fig_3.data[3].y = B(x, L=L, z_a=z_a, z_b=z_b)
        x, dx = np.linspace(0, L, n, retstep=True)
        y_R, dydx_R = R(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        y_B, dydx_B = B(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        fig_3.data[1].x = x[:-1]
        fig_3.data[1].y = y_R[:-1]
        fig_3.data[1].width = dx
        fig_3.data[0].x = x[:-1]
        fig_3.data[0].y = y_B[:-1]
        fig_3.data[0].width = dx
        fig_3.data[-1].x = [0, L]
        fig_3.data[-1].y = [z_a, z_b]
        fig_3.data[-1].uid = str(uuid.uuid4())  # This forces the text to be updated
        travail_R = travail(x, y_R, dydx_R, force(x, y_R))
        travail_B = travail(x, y_B, dydx_B, force(x, y_B))
        fig_3.layout.title.text = (
            f"Cas conservatif - Travail R : {travail_R:.4f} / Travail B : {travail_B:.4f}"
        )


fig_3

<hr>

# 5. Travail d'une force, dans un cas non conservatif

Dans la vraie vie, les champs de force sont rarement conservatifs ! Ici, nous vous proposons de modéliser un champ de force non conservatif en rajouter une composante horizontale (due au vent) qui pousse notre skieur vers la droite. Cette composante est d'autant plus élevée que le skieur est en altitude :

$$ \vec{F}(x, y) = \begin{pmatrix}\alpha \cdot y & -m\cdot g\end{pmatrix}^\intercal.$$

La cellule suivante définit une telle force. Vous pouvez directement exécuter (<i class="fa-step-forward fa"></i>) les deux prochaines cellules pour voir l'effet de cette nouvelle composante sur le travail total.

> NOTE : vous êtes encouragés à tester d'autres champs de force en changeant le code ci-dessous, et à observer l'impact sur le travail.

In [ ]:
def force_non_conservatrice(x: np.ndarray, y: np.ndarray, alpha: float = 20) -> np.ndarray:
    """
    Evalue le vecteur force appliqué à un ou plusieurs points (x, y).

    x: vecteur abscisses, de 0 à L, de longueur n
    y: vecteur ordonnées, de longueur n
    alpha: coefficient de prise au vent
    """
    g = 9.81  # m / s^2
    m = 1  # kg 
    n = len(x)

    # Imaginez que le vent nous pousse + fort si on est + haut
    F_x = alpha * y  # cette ligne a changé ;-)
    F_y = -m * np.ones(n) * g
    return np.column_stack([F_x, F_y])

In [ ]:
trace_R = go.Scatter(
    x=[],
    y=[],
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=[],
    y=[],
    name="B",
    mode="lines",
    line_color="blue",
)

trace_R_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="red",
    name="Aire approx. de R",
    opacity=0.5,
)

trace_B_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="blue",
    name="Aire approx. de B",
    opacity=0.5,
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_4 = go.FigureWidget(
    data=[trace_B_approx, trace_R_approx, trace_R, trace_B, trace_Z]
)


@interact(
    L=(1, 10, 1),
    z_a=(0, 100, 1),
    z_b=(0, 100, 1),
    n=(4, 100, 4),
    alpha_r=(0, 100, 4),
    alpha_b=(0, 100, 4),
)
def update(L=1, z_a=1, z_b=0, n=4, alpha_r=20, alpha_b=20):
    with fig_4.batch_update():
        x = np.linspace(0, L)
        fig_4.data[2].x = x
        fig_4.data[2].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_4.data[3].x = x
        fig_4.data[3].y = B(x, L=L, z_a=z_a, z_b=z_b)
        x, dx = np.linspace(0, L, n, retstep=True)
        y_R, dydx_R = R(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        y_B, dydx_B = B(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        fig_4.data[1].x = x[:-1]
        fig_4.data[1].y = y_R[:-1]
        fig_4.data[1].width = dx
        fig_4.data[0].x = x[:-1]
        fig_4.data[0].y = y_B[:-1]
        fig_4.data[0].width = dx
        fig_4.data[-1].x = [0, L]
        fig_4.data[-1].y = [z_a, z_b]
        fig_4.data[-1].uid = str(uuid.uuid4())  # This forces the text to be updated
        travail_R = travail(
            x, y_R, dydx_R, force_non_conservatrice(x, y_R, alpha=alpha_r)
        )
        travail_B = travail(
            x, y_B, dydx_B, force_non_conservatrice(x, y_B, alpha=alpha_b)
        )
        fig_4.layout.title.text = f"Cas non conservatif - Travail R : {travail_R:.4f} / Travail B : {travail_B:.4f}"


fig_4

<hr>

# 6. Pour aller plus loin

La matière de ce Notebook s'arrête ici. Cependant, vous verrez dans vos cours que ce calcul d'intégrale le long d'une courbe (intégrale curviligne) a de nombreuses applications, et pas seulement en 2D !

Bien sûr, il est également possible de généraliser le concept d'intégration numérique à 3+ dimensions, mais ça sera pour la prochaine fois ;-).

<hr>

Code intégralement disponible ici : https://github.com/jeertmans/LEPL1202_APP1-B.